In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import date
from operator import attrgetter
import numpy as np
warnings.filterwarnings("ignore")

In [2]:
loitering = pd.read_csv('loitering.csv')
encounters = pd.read_csv('encounter.csv')
port_visits = pd.read_csv('port_visits.csv')
carriers = pd.read_csv('carriers.csv')

In [3]:
id = carriers[['mmsi','imo','dataset']].copy()
id[id.dataset == 'API-port']
id = id.drop_duplicates(subset=['mmsi'])
id = id.set_index('mmsi')
id = id.fillna(-1)
id = id.join(loitering.groupby(['vessel.mmsi'])['id'].count())
id = id.rename({
    'id': 'loitering',
    },
    axis = 1)

In [4]:
# loitering.groupby(['vessel.mmsi'])['id'].count().reset_index()

In [5]:
#loitering and encounter number with types
a = encounters.groupby(['vessel.mmsi','encounter.authorization_status'])['id'].count().reset_index()
b = encounters.groupby(['encounter.encountered_vessel.mmsi','encounter.authorization_status'])['id'].count().reset_index()
a = a.pivot(index='vessel.mmsi', columns='encounter.authorization_status', values='id')
b = b.pivot(index='encounter.encountered_vessel.mmsi', columns='encounter.authorization_status', values='id')
tmp = pd.concat([a, b]).reset_index().groupby(['index'])['authorized','partial','pending','unknown'].sum()
tmp = tmp.rename({
    'authorized': 'authorized_encounter',
    'partial': 'partial_encounter',
    'pending': 'pending_encounter',
    'unknown': 'unknown_encounter',
    },
    axis = 1)

id2 = id.join(tmp)
id2['encounter'] = id2['authorized_encounter'] + id2['partial_encounter'] + id2['pending_encounter'] + id2['unknown_encounter']
id2 = id2.fillna(0)

In [6]:
# id2

In [7]:
# plt.scatter(id2['loitering'],id2['unknown_encounter'])

In [8]:
# # (num of encounter)/(num of encounter + num of loitering)
# # delete the 0 encounter and 0 loitering
# tmp = (id2['encounter']/(id2['loitering']+id2['encounter']))
# sns.distplot(tmp.loc[lambda x : (x < 1) & (x > 0)], hist=True, kde=True, 
#              bins=int(180/5), color = 'darkblue', 
#              hist_kws={'edgecolor':'black'},
#              kde_kws={'linewidth': 4})

In [9]:
# last 6 month and 12 month
loitering['month'] = pd.to_datetime(loitering['start'].str.replace('UTC','')).dt.to_period('M')
encounters['month'] = pd.to_datetime(encounters['start'].str.replace('UTC','')).dt.to_period('M')
l = loitering[loitering['month'] >= loitering['month'].max()-6]
e = encounters[encounters['month'] >= encounters['month'].max()-6]

id3 = id2.join(l.groupby(['vessel.mmsi'])['id'].count())
id3 = id3.rename({
    'id': 'loitering_06',
    },
    axis = 1)
id3 = id3.join(e.groupby(['vessel.mmsi'])['id'].count())
id3 = id3.rename({
    'id': 'encounters_06',
    },
    axis = 1)
l = loitering[loitering['month'] >= loitering['month'].max()-12]
e = encounters[encounters['month'] >= encounters['month'].max()-12]
id3 = id3.join(l.groupby(['vessel.mmsi'])['id'].count())
id3 = id3.rename({
    'id': 'loitering_12',
    },
    axis = 1)
id3 = id3.join(e.groupby(['vessel.mmsi'])['id'].count())
id3 = id3.rename({
    'id': 'encounters_12',
    },
    axis = 1)
id3 = id3.fillna(0)
# topmmsi = l.groupby(['vessel.mmsi',])['id'].count().nlargest(3).index

# tmp = loitering.groupby(['vessel.mmsi','month'])['id'].count()
# tmp = tmp[topmmsi].reset_index()
# tmp = tmp[tmp['month'] >= '2022-01']
# tmp['month'] = tmp['month'].astype(str)

# append a column of vessels that have highest meet-ups during last n months.

In [10]:
# sns.relplot(data=tmp, kind='line',x='month', y='id', hue='vessel.mmsi')

In [11]:
# loitering.groupby(['month'])['id'].count().plot()

In [12]:
# port_visits['month']

In [13]:
# most frequent port_visit 
visit=port_visits.groupby(['vessel_ssvid','name','flag'])['event_id'].count().sort_values().groupby(level=0).tail(1)
visit
id4 = id3.join(visit.reset_index().set_index('vessel_ssvid'))
id4 = id4.rename({
    'name': 'port_name',
    'event_id': 'visit_times',
    'flag':'port_country'
    },
    axis = 1)
id4

,imo,dataset,loitering,authorized_encounter,partial_encounter,pending_encounter,unknown_encounter,encounter,loitering_06,encounters_06,loitering_12,encounters_12,port_name,port_country,visit_times
mmsi,,,,,,,,,,,,,,,
566041000,9437186.0,public-global-carrier-vessels:v20201001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DAKAR,SEN,56.0
538003418,8617926.0,public-global-carrier-vessels:v20201001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,COLON,PAN,40.0
273435310,0.0,public-global-carrier-vessels:v20201001,83.0,0.0,0.0,0.0,83.0,83.0,0.0,0.0,0.0,0.0,BELLSUND,SJM,42.0
431600125,-1.0,public-global-carrier-vessels:v20201001,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,HIRADO,JPN,99.0
273815000,0.0,public-global-carrier-vessels:v20201001,115.0,1.0,0.0,1.0,113.0,115.0,12.0,13.0,35.0,27.0,VLADIVOSTOK,RUS,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412549394,-1.0,API-port,0.0,0.0,0.0,0.0,7.0,7.0,0.0,0.0,0.0,0.0,SINGAPORE,SGP,1.0
412549391,-1.0,API-port,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,SINGAPORE,SGP,1.0
412549392,-1.0,API-port,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,SINGAPORE,SGP,1.0


In [14]:
# last port_visit
port_visits.groupby(['vessel_ssvid'])['event_id'].count()
port_visits['month'] = pd.to_datetime(port_visits['event_start'].str.replace('UTC','')).dt.to_period('D')
last = port_visits.groupby(['vessel_ssvid','name','flag'])['month'].max().sort_values().groupby(level=0).tail(1)

id5 = id4.join(last.reset_index().set_index('vessel_ssvid'))

id5 = id5.rename({
    'name': 'last_port_name',
    'month': 'last_visit_times',
    'flag' : 'last_port_country'
    },
    axis = 1)
id5

,imo,dataset,loitering,authorized_encounter,partial_encounter,pending_encounter,unknown_encounter,encounter,loitering_06,encounters_06,loitering_12,encounters_12,port_name,port_country,visit_times,last_port_name,last_port_country,last_visit_times
mmsi,,,,,,,,,,,,,,,,,,
566041000,9437186.0,public-global-carrier-vessels:v20201001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DAKAR,SEN,56.0,ALGECIRAS,ESP,2022-10-26
538003418,8617926.0,public-global-carrier-vessels:v20201001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,COLON,PAN,40.0,BHAVNAGAR,IND,2017-06-08
273435310,0.0,public-global-carrier-vessels:v20201001,83.0,0.0,0.0,0.0,83.0,83.0,0.0,0.0,0.0,0.0,BELLSUND,SJM,42.0,VOLGO-KASPIYSKIY,RUS,2022-04-06
431600125,-1.0,public-global-carrier-vessels:v20201001,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,HIRADO,JPN,99.0,HIRADO,JPN,2021-03-21
273815000,0.0,public-global-carrier-vessels:v20201001,115.0,1.0,0.0,1.0,113.0,115.0,12.0,13.0,35.0,27.0,VLADIVOSTOK,RUS,47.0,VLADIVOSTOK,RUS,2022-10-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412549394,-1.0,API-port,0.0,0.0,0.0,0.0,7.0,7.0,0.0,0.0,0.0,0.0,SINGAPORE,SGP,1.0,SINGAPORE,SGP,2021-09-05
412549391,-1.0,API-port,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,SINGAPORE,SGP,1.0,SINGAPORE,SGP,2021-09-05
412549392,-1.0,API-port,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,SINGAPORE,SGP,1.0,SINGAPORE,SGP,2021-09-05


In [15]:
# id5.columns

In [16]:
id5['total_meetup'] = id5[['loitering', 'authorized_encounter',
       'partial_encounter', 'pending_encounter', 'unknown_encounter']].sum(axis=1)

In [17]:

id5['imo'] = id5['imo'].replace(0, np.nan).replace(-1, np.nan)

In [18]:
id5['average_meetup'] = port_visits.groupby(['vessel_ssvid'])['event_id'].count()

In [19]:
id5['average_meetup'] = id5['total_meetup']/id5['average_meetup']
id5.reset_index().rename({
    'index': 'MMSI',
    },
    axis = 1)

## The meetup likelihood
port_visits['month'] = pd.to_datetime(port_visits['event_start'].str.replace('UTC','')).dt.to_period('M')
id5['last_port_visit_month'] = port_visits.groupby(['vessel_ssvid'])['month'].max().to_frame()
id5['avg_month_port_visit'] = encounters[['vessel.mmsi', 'month']].groupby(['vessel.mmsi', 'month'])['month'].count() \
    .to_frame().rename({'month': 'count',},axis = 1) \
    .add(encounters[['encounter.encountered_vessel.mmsi', 'month']] \
         .groupby(['encounter.encountered_vessel.mmsi', 'month'])['month'].count() \
         .to_frame().rename({'month': 'count',},axis = 1) \
         .reset_index().rename({'encounter.encountered_vessel.mmsi': 'vessel.mmsi',},axis = 1) \
         .set_index(['vessel.mmsi','month']), fill_value=0) \
    .add(loitering[['vessel.mmsi', 'month']] \
         .groupby(['vessel.mmsi', 'month'])['month'].count() \
         .to_frame().rename({'month': 'count',},axis = 1), fill_value=0).reset_index() \
.groupby(['vessel.mmsi'])['count'].mean().to_frame()
id5['meetup_since_last_port'] = 0
id5.loc[566041000]['meetup_since_last_port'] = 1

m = []
for i in id5.index:
    ttl = 0
    ttl += loitering[(loitering['vessel.mmsi'] == i) & (loitering['month']>=id5.loc[i]['last_port_visit_month'])]['id'].count()
    ttl += encounters[(encounters['vessel.mmsi'] == i) & (encounters['month']>=id5.loc[i]['last_port_visit_month'])]['id'].count()
    ttl += encounters[(encounters['encounter.encountered_vessel.mmsi'] == i) & (encounters['month']>=id5.loc[i]['last_port_visit_month'])]['id'].count()
    m.append(ttl)
m
id5['meetup_since_last_port'] = m
id5['month_since_last_port'] = (pd.Period(date.today(), freq='M').to_timestamp()-id5['last_port_visit_month'].dt.to_timestamp())//np.timedelta64(1, 'M')
id5['likely_remaining_meetup'] = id5['month_since_last_port']*id5['avg_month_port_visit'] - id5['meetup_since_last_port']

In [20]:
id5

,imo,dataset,loitering,authorized_encounter,partial_encounter,pending_encounter,unknown_encounter,encounter,loitering_06,encounters_06,...,last_port_name,last_port_country,last_visit_times,total_meetup,average_meetup,last_port_visit_month,avg_month_port_visit,meetup_since_last_port,month_since_last_port,likely_remaining_meetup
mmsi,,,,,,,,,,,,,,,,,,,,,
566041000,9437186.0,public-global-carrier-vessels:v20201001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,ALGECIRAS,ESP,2022-10-26,0.0,0.000000,2022-10,NaN,0,1.0,NaN
538003418,8617926.0,public-global-carrier-vessels:v20201001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,BHAVNAGAR,IND,2017-06-08,0.0,0.000000,2017-06,NaN,0,65.0,NaN
273435310,NaN,public-global-carrier-vessels:v20201001,83.0,0.0,0.0,0.0,83.0,83.0,0.0,0.0,...,VOLGO-KASPIYSKIY,RUS,2022-04-06,166.0,0.597122,2022-10,6.148148,0,1.0,6.148148
431600125,NaN,public-global-carrier-vessels:v20201001,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,...,HIRADO,JPN,2021-03-21,4.0,0.013072,2021-07,2.000000,0,16.0,32.000000
273815000,NaN,public-global-carrier-vessels:v20201001,115.0,1.0,0.0,1.0,113.0,115.0,12.0,13.0,...,VLADIVOSTOK,RUS,2022-10-11,230.0,1.464968,2022-10,8.250000,0,1.0,8.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412549394,NaN,API-port,0.0,0.0,0.0,0.0,7.0,7.0,0.0,0.0,...,SINGAPORE,SGP,2021-09-05,7.0,7.000000,2021-09,1.333333,8,13.0,9.333333
412549391,NaN,API-port,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,...,SINGAPORE,SGP,2021-09-05,5.0,5.000000,2021-09,1.200000,6,13.0,9.600000
412549392,NaN,API-port,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,...,SINGAPORE,SGP,2021-09-05,5.0,5.000000,2021-09,1.200000,6,13.0,9.600000


In [21]:
# encounters[['vessel.mmsi', 'eez']].groupby(['vessel.mmsi', 'eez'])['eez'].count()

In [22]:
# encounters[['regions.rfmo']] = encounters[['regions.rfmo']].to_string().split('|')

In [23]:
# for i in range(len(encounters[['regions.rfmo']])):
#     encounters[['regions.rfmo']].iloc[i,:] = encounters[['regions.rfmo']].iloc[i,:].replace('|',' ').replace(',',' ').str.split()

In [24]:
# encounters[['regions.rfmo']]

In [25]:
# encounters[['regions.rfmo']].iloc[0,:] = encounters[['regions.rfmo']].iloc[0,:].replace('|',' ').replace(',',' ').str.split()

In [26]:
# encounters[['regions.rfmo']]

In [27]:
additional = {
    '8487': 'Japanese Exclusive Economic Zone',
    '8492': 'Indonesian Exclusive Economic Zone',
    '48950': 'Kuril Islands (Japan v. Russia)',
    '8471': 'Guinea Bissau Exclusive Economic Zone',
    '8463': 'United States Exclusive Economic Zone (Alaska)',
    '8488': 'Kiribati Exclusive Economic Zone',
    '8486': 'Chinese Exclusive Economic Zone',
    '8493': 'Canadian Exclusive Economic Zone',
}
loitering.loc[loitering.eez =='48964, 8471','eez'] = '8471'
loitering.loc[loitering.eez =='48962, 8431','eez'] = '8431'
loitering.loc[loitering.eez =='48967, 5696','eez'] = '5696'
loitering.loc[loitering.eez =='8435, 48967','eez'] = '8435'
loitering.loc[loitering.eez =='8467, 48961','eez'] = '8467'
loitering.loc[loitering.eez =='48967, 8435','eez'] = '8435'
loitering.loc[loitering.eez =='8431, 48962','eez'] = '8431'
loitering.loc[loitering.eez =='8371, 48964','eez'] = '8371'
loitering.loc[loitering.eez =='5696, 48967','eez'] = '5696'
loitering.loc[loitering.eez =='8471, 48964','eez'] = '8471'
loitering.loc[loitering.eez =='48969, 8418','eez'] = '8418'
loitering.loc[loitering.eez =='48964, 8371','eez'] = '8371'
loitering.loc[loitering.eez =='48961, 8467','eez'] = '8467'
loitering.loc[loitering.eez =='48974, 8431','eez'] = '8431'
loitering.loc[loitering.eez =='48962, 8432','eez'] = '8432'
loitering.loc[loitering.eez =='8431, 48974','eez'] = '8431'
loitering.loc[loitering.eez =='48970, 8426','eez'] = '8426'
loitering.eez = loitering.eez.astype('float64')
encounters.eez =encounters.eez.astype('float64')

In [28]:
# mmsi + eez
eez = encounters[['vessel.mmsi', 'eez']].groupby(['vessel.mmsi', 'eez'])['eez'].count() \
    .to_frame().rename({'eez': 'count',},axis = 1) \
    .add(encounters[['encounter.encountered_vessel.mmsi', 'eez']] \
         .groupby(['encounter.encountered_vessel.mmsi', 'eez'])['eez'].count() \
         .to_frame().rename({'eez': 'count',},axis = 1) \
         .reset_index().rename({'encounter.encountered_vessel.mmsi': 'vessel.mmsi',},axis = 1) \
         .set_index(['vessel.mmsi','eez']), fill_value=0) \
    .add(loitering[['vessel.mmsi', 'eez']] \
         .groupby(['vessel.mmsi', 'eez'])['eez'].count() \
         .to_frame().rename({'eez': 'eez_visit',},axis = 1), fill_value=0).reset_index()
eez.eez = eez.eez.astype('int').astype('str')
eez = eez[eez['eez'].isin(additional)].set_index(['vessel.mmsi','eez']).unstack().rename(additional,
    axis = 1)
eez

count  \
eez         Kuril Islands (Japan v. Russia)   
vessel.mmsi                                   
5004                                    NaN   
200019507                               NaN   
200042177                               NaN   
259568000                               NaN   
273213710                               3.0   
...                                     ...   
613003558                               NaN   
630123900                               NaN   
630124004                               NaN   
636018227                               NaN   
700000777                               NaN   

                                                            \
eez         United States Exclusive Economic Zone (Alaska)   
vessel.mmsi                                                  
5004                                                   2.0   
200019507                                              NaN   
200042177                                              NaN   
259568000                                              NaN   
273213710                                              NaN   
...                                                    ...   
613003558                                              NaN   
630123900                                              NaN   
630124004                                              NaN   
636018227                                              NaN   
700000777                                              NaN   

                                                   \
eez         Guinea Bissau Exclusive Economic Zone   
vessel.mmsi                                         
5004                                          NaN   
200019507                                     NaN   
200042177                                     NaN   
259568000                                     NaN   
273213710                                     NaN   
...                                           ...   
613003558                                     2.0   
630123900                                     1.0   
630124004                                     1.0   
636018227                                     NaN   
700000777                                     NaN   

                                                                              \
eez         Chinese Exclusive Economic Zone Japanese Exclusive Economic Zone   
vessel.mmsi                                                                    
5004                                    NaN                              NaN   
200019507                               1.0                              NaN   
200042177                               2.0                              NaN   
259568000                               NaN                              NaN   
273213710                               NaN                              NaN   
...                                     ...                              ...   
613003558                               NaN                              NaN   
630123900                               NaN                              NaN   
630124004                               NaN                              NaN   
636018227                               1.0                              NaN   
700000777                               3.0                              NaN   

                                              \
eez         Kiribati Exclusive Economic Zone   
vessel.mmsi                                    
5004                                     NaN   
200019507                                NaN   
200042177                                NaN   
259568000                                NaN   
273213710                                NaN   
...                                      ...   
613003558                                NaN   
630123900                                NaN   
630124004                                NaN   
636018227                                NaN   
700000777             

In [29]:
id6 = id5.join(eez)

In [30]:
# loitering \
#     .rename({'regions.rfmo': 'rfmo'}, axis=1) \
#     .groupby('rfmo')['vessel.mmsi'].nunique()

In [31]:
id6

,imo,dataset,loitering,authorized_encounter,partial_encounter,pending_encounter,unknown_encounter,encounter,loitering_06,encounters_06,...,"(count, Indonesian Exclusive Economic Zone)","(count, Canadian Exclusive Economic Zone)","(eez_visit, Kuril Islands (Japan v. Russia))","(eez_visit, United States Exclusive Economic Zone (Alaska))","(eez_visit, Guinea Bissau Exclusive Economic Zone)","(eez_visit, Chinese Exclusive Economic Zone)","(eez_visit, Japanese Exclusive Economic Zone)","(eez_visit, Kiribati Exclusive Economic Zone)","(eez_visit, Indonesian Exclusive Economic Zone)","(eez_visit, Canadian Exclusive Economic Zone)"
mmsi,,,,,,,,,,,,,,,,,,,,,
566041000,9437186.0,public-global-carrier-vessels:v20201001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
538003418,8617926.0,public-global-carrier-vessels:v20201001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273435310,NaN,public-global-carrier-vessels:v20201001,83.0,0.0,0.0,0.0,83.0,83.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431600125,NaN,public-global-carrier-vessels:v20201001,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273815000,NaN,public-global-carrier-vessels:v20201001,115.0,1.0,0.0,1.0,113.0,115.0,12.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412549394,NaN,API-port,0.0,0.0,0.0,0.0,7.0,7.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412549391,NaN,API-port,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412549392,NaN,API-port,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
plot_data5 = loitering \
    .rename({'regions.rfmo': 'rfmo'}, axis=1) \
    .groupby(['rfmo','vessel.mmsi'])['id'] \
    .nunique() \
    .reset_index() \
    .sort_values('id')
plot_data5 = plot_data5.set_index(plot_data5.columns.drop('rfmo',1).tolist()) \
    .rfmo.str.split(',', expand=True) \
    .stack() \
    .reset_index() \
    .rename(columns={0:'rfmo'}) \
    .loc[:, plot_data5.columns] \
    .groupby(['rfmo','vessel.mmsi'])['id'] \
    .sum() \
    .reset_index() \
    .sort_values('id')
plot_data5 = plot_data5.set_index(plot_data5.columns.drop('rfmo',1).tolist()) \
    .rfmo.str.split('|', expand=True) \
    .stack() \
    .reset_index() \
    .rename(columns={0:'rfmo'}) \
    .loc[:, plot_data5.columns] \
    .groupby(['rfmo','vessel.mmsi'])['id'] \
    .sum() \
    .reset_index() \
    .sort_values('id')
plot_data5['rfmo'] = plot_data5['rfmo'].str.replace(' ','')
plot_data5 = plot_data5 \
    .groupby(['vessel.mmsi','rfmo'])['id'] \
    .sum() \
    .reset_index() \
    .sort_values('id').rename({'id': 'rfmo_visit'}, axis=1)

rfmo = plot_data5.set_index(['vessel.mmsi','rfmo']).unstack()
id6 = id6.join(rfmo)
# .to_csv('carrier_mmsi.csv')


,imo,dataset,loitering,authorized_encounter,partial_encounter,pending_encounter,unknown_encounter,encounter,loitering_06,encounters_06,...,"(rfmo_visit, NPFC)","(rfmo_visit, PICES)","(rfmo_visit, RECOFI)","(rfmo_visit, SEAFDEC)","(rfmo_visit, SIOFA)","(rfmo_visit, SPC)","(rfmo_visit, SPRFMO)","(rfmo_visit, SRFC)","(rfmo_visit, SWIOFC)","(rfmo_visit, WCPFC)"
mmsi,,,,,,,,,,,,,,,,,,,,,
566041000,9437186.0,public-global-carrier-vessels:v20201001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
538003418,8617926.0,public-global-carrier-vessels:v20201001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273435310,NaN,public-global-carrier-vessels:v20201001,83.0,0.0,0.0,0.0,83.0,83.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431600125,NaN,public-global-carrier-vessels:v20201001,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
273815000,NaN,public-global-carrier-vessels:v20201001,115.0,1.0,0.0,1.0,113.0,115.0,12.0,13.0,...,NaN,113.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412549394,NaN,API-port,0.0,0.0,0.0,0.0,7.0,7.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412549391,NaN,API-port,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412549392,NaN,API-port,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
id6 = id6.join(rfmo)
id6.columns

Index([                                                          'imo',
                                                             'dataset',
                                                           'loitering',
                                                'authorized_encounter',
                                                   'partial_encounter',
                                                   'pending_encounter',
                                                   'unknown_encounter',
                                                           'encounter',
                                                        'loitering_06',
                                                       'encounters_06',
                                                        'loitering_12',
                                                       'encounters_12',
                                                           'port_name',
                                                        'port_co

In [34]:
plot_data5 = loitering \
    .rename({'regions.rfmo': 'rfmo'}, axis=1) \
    .groupby(['rfmo','vessel.mmsi'])['id'] \
    .nunique() \
    .reset_index() \
    .sort_values('id')


In [35]:
# loitering.groupby(['vessel.mmsi'])['id'].count().sort_values()

In [36]:
# encounters[encounters['vessel.mmsi']==273812710]

In [39]:
test =id5.reset_index().rename({
    'index': 'MMSI',
    },
    axis = 1)
# test[test['MMSI'] == 273376120]

In [ ]:
from typing import Union
def foo(x : Union[int, list]) -> Union[int, list]:
    return 1
foo({'1':1})

In [ ]:
loitering.groupby(['vessel.mmsi','month'])['id'].count().reset_index

In [ ]:
port_visits.groupby(['vessel_ssvid','month'])['event_id'].count()

In [ ]:
encounters[['vessel.mmsi', 'month']].groupby(['vessel.mmsi', 'month'])['month'].count() \
    .to_frame().rename({'month': 'count',},axis = 1) \
    .add(encounters[['encounter.encountered_vessel.mmsi', 'month']] \
         .groupby(['encounter.encountered_vessel.mmsi', 'month'])['month'].count() \
         .to_frame().rename({'month': 'count',},axis = 1) \
         .reset_index().rename({'encounter.encountered_vessel.mmsi': 'vessel.mmsi',},axis = 1) \
         .set_index(['vessel.mmsi','month']), fill_value=0) \
    .add(loitering[['vessel.mmsi', 'month']] \
         .groupby(['vessel.mmsi', 'month'])['month'].count() \
         .to_frame().rename({'month': 'count',},axis = 1), fill_value=0).reset_index() \
.groupby(['vessel.mmsi'])['count'].mean().to_frame()

In [ ]:
port_visits.groupby(['vessel_ssvid'])['month'].max().to_frame()

In [ ]:
## The meetup likelihood
id5['last_port_visit_month'] = port_visits.groupby(['vessel_ssvid'])['month'].max().to_frame()
id5['avg_month_port_visit'] = encounters[['vessel.mmsi', 'month']].groupby(['vessel.mmsi', 'month'])['month'].count() \
    .to_frame().rename({'month': 'count',},axis = 1) \
    .add(encounters[['encounter.encountered_vessel.mmsi', 'month']] \
         .groupby(['encounter.encountered_vessel.mmsi', 'month'])['month'].count() \
         .to_frame().rename({'month': 'count',},axis = 1) \
         .reset_index().rename({'encounter.encountered_vessel.mmsi': 'vessel.mmsi',},axis = 1) \
         .set_index(['vessel.mmsi','month']), fill_value=0) \
    .add(loitering[['vessel.mmsi', 'month']] \
         .groupby(['vessel.mmsi', 'month'])['month'].count() \
         .to_frame().rename({'month': 'count',},axis = 1), fill_value=0).reset_index() \
.groupby(['vessel.mmsi'])['count'].mean().to_frame()
id5['meetup_since_last_port'] = 0
id5.loc[566041000]['meetup_since_last_port'] = 1

m = []
for i in id5.index:
    ttl = 0
    ttl += loitering[(loitering['vessel.mmsi'] == i) & (loitering['month']>=id5.loc[i]['last_port_visit_month'])]['id'].count()
    ttl += encounters[(encounters['vessel.mmsi'] == i) & (encounters['month']>=id5.loc[i]['last_port_visit_month'])]['id'].count()
    ttl += encounters[(encounters['encounter.encountered_vessel.mmsi'] == i) & (encounters['month']>=id5.loc[i]['last_port_visit_month'])]['id'].count()
    m.append(ttl)
m
id5['meetup_since_last_port'] = m
id5['month_since_last_port'] = (pd.Period(date.today(), freq='M').to_timestamp()-id5['last_port_visit_month'].dt.to_timestamp())//np.timedelta64(1, 'M')
id5['likely_remaining_meetup'] = id5['month_since_last_port']*id5['avg_month_port_visit'] - id5['meetup_since_last_port']

In [ ]:
id5

In [ ]:

id5.loc[id5.index[0]]['last_port_visit_month']
loitering[(loitering['vessel.mmsi'] == id5.index[0]) & (loitering['month']>=id5.loc[id5.index[0]]['last_port_visit_month'])]['id'].count()

In [ ]:
id5.loc[id5.index[0]]['last_port_visit_month']

In [ ]:
encounters[(encounters['vessel.mmsi'] == id5.index[0]) & (encounters['month']>=id5.loc[id5.index[0]]['last_port_visit_month'])]['id'].count()

In [ ]:
m = []
for i in id5.index:
    ttl = 0
    ttl += loitering[(loitering['vessel.mmsi'] == i) & (loitering['month']>=id5.loc[i]['last_port_visit_month'])]['id'].count()
    ttl += encounters[(encounters['vessel.mmsi'] == i) & (encounters['month']>=id5.loc[i]['last_port_visit_month'])]['id'].count()
    ttl += encounters[(encounters['encounter.encountered_vessel.mmsi'] == i) & (encounters['month']>=id5.loc[i]['last_port_visit_month'])]['id'].count()
    m.append(ttl)
m
id5['meetup_since_last_port'] = m
id5['month_since_last_port'] = (pd.Period(date.today(), freq='M').to_timestamp()-id5['last_port_visit_month'].dt.to_timestamp())//np.timedelta64(1, 'M')
id5['likely_remaining_meetup'] = id5['month_since_last_port']*id5['avg_month_port_visit'] - id5['meetup_since_last_port']

In [ ]:
id5

In [ ]:
id5['month_since_last_port']*id5['avg_month_port_visit'] - id5['meetup_since_last_port']

In [ ]:
pd.to_datetime(port_visits['event_start'].str.replace('UTC','')).dt.to_period('M')